Εργασία 2: Ταξινόμηση για **OCR**

In [1]:
#Import the basic packages (we will import some more below)
from sklearn import datasets # Library for machine learning
import numpy as np # Library for math and arrays
np.set_printoptions(precision=3) # Tell numpy to print float numbers with 3-decimal precision
import matplotlib.pyplot as plt # Library for graphs and plots
from IPython.display import display, Markdown
from sklearn import preprocessing

#Markdown cheat sheet: https://www.markdownguide.org/cheat-sheet/

Άνοιγμα αρχείου και προεργασία

In [2]:
import csv
results = []
Y = []
X = []

with open('letter-recognition.csv') as csvfile:
        data = csv.reader(csvfile, delimiter=',')       #Δημιουργείται έτσι μια λίστα, κάθε στοιχείο της οποίας είναι επίσης λίστα,
        for dataline in data:
              results.append(dataline)
print(results)
print("\n")
print("-------------------------------------------------------------------")
print("\n")

#μετατρέpουμε μια λίστα σε πίνακα numpy
myArray = np.array(results)

# ftiaxnw tis nees listes. Stin prwth afairw thn prvth thesi kai sthn defterpairnw to teleftaiw stoixio
X = myArray[:,1:-1]
Y = myArray[:,0]
  
print(X)
print("\n")
print(Y)
print("-------------------------------------------------------------------")
print("\n")

#μετατροπή ενός numpy πίνακα από strings σε έναν numpy πίνακα από floats 
New_X = X.astype(float)

#Metatroph tou y se alpharitmhmitiko
le = preprocessing.LabelEncoder()
lel = le.fit(Y)
New_Y = lel.transform(Y)

#Δημιουργw τους πίνακες x_train, y_train, x_test, y_test
X_train, X_test = New_X[:16000], New_X[16000:]
y_train, y_test = New_Y[:16000], New_Y[16000:]

#Τυπώστε το μέσο, τη διασπορά, το μέγιστο και το ελάχιστο κάθε στήλης του x_train και του x_test.
print("Average X_train:",np.mean(X_train, axis=0))
print("Average X_test:",np.mean(X_test, axis=0))
print("Variance X_train:",np.var(X_train, axis=0))
print("Variance X_test:",np.var(X_test, axis=0))
print("MAX X_train:",X_train.max(axis=0))
print("MAX X_test:",X_test.max(axis=0))
print("MIN X_train:",X_train.min(axis=0))
print("MIN X_test:",X_test.min(axis=0))
print("\n")

[['T', '2', '8', '3', '5', '1', '8', '13', '0', '6', '6', '10', '8', '0', '8', '0', '8'], ['I', '5', '12', '3', '7', '2', '10', '5', '5', '4', '13', '3', '9', '2', '8', '4', '10'], ['D', '4', '11', '6', '8', '6', '10', '6', '2', '6', '10', '3', '7', '3', '7', '3', '9'], ['N', '7', '11', '6', '6', '3', '5', '9', '4', '6', '4', '4', '10', '6', '10', '2', '8'], ['G', '2', '1', '3', '1', '1', '8', '6', '6', '6', '6', '5', '9', '1', '7', '5', '10'], ['S', '4', '11', '5', '8', '3', '8', '8', '6', '9', '5', '6', '6', '0', '8', '9', '7'], ['B', '4', '2', '5', '4', '4', '8', '7', '6', '6', '7', '6', '6', '2', '8', '7', '10'], ['A', '1', '1', '3', '2', '1', '8', '2', '2', '2', '8', '2', '8', '1', '6', '2', '7'], ['J', '2', '2', '4', '4', '2', '10', '6', '2', '6', '12', '4', '8', '1', '6', '1', '7'], ['M', '11', '15', '13', '9', '7', '13', '2', '6', '2', '12', '1', '9', '8', '1', '1', '8'], ['X', '3', '9', '5', '7', '4', '8', '7', '3', '8', '5', '6', '8', '2', '8', '6', '7'], ['O', '6', '13', '4'

Ταξινόμηση

In [3]:
#Λογιστική παλινδρόμηση με Pipeline


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# Two steps: a scaler and then a classifier. Declared as tuples
log_reg_pipeline=Pipeline([('Scaler',  StandardScaler()),
            ('LogRegClassifier', LogisticRegression(max_iter=1000))])

#IMPORTANT: the parameter name must be <name of step>__<name of parameter>
params = { 
           'LogRegClassifier__C': np.logspace(-2, 3, 6) # try 6 values in logarithmic scale from 10^-2 to 10^3
         }

grid = GridSearchCV(estimator=log_reg_pipeline, cv = 5, param_grid=params, verbose=4)
grid.fit(X_train,y_train)

y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

display(Markdown(
    "### Αποτελέσματα λογιστικής παλινδρόμησης:\n" +
    "**Best lambda**: {:.3f}".format(grid.best_params_['LogRegClassifier__C'])  + "<br>" +
    "**Best Cross-Val Accuracy**: {:.2f}".format(grid.best_score_) + "<br>" +
    "**Test Set Accuracy**: {:.2f}".format(accuracy_score(y_test, y_test_pred)) + "<br>" +
    "**Test Set F1**: {:.2f}".format(f1_score(y_test, y_test_pred,average='micro')) + "<br>" +
    "**Confusion matrix**:"
))
print("\n".join([' '.join(['{:4}'.format(item) for item in row]) 
      for row in confusion_matrix(y_test,y_test_pred)]))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ..........LogRegClassifier__C=0.01;, score=0.710 total time=   2.8s
[CV 2/5] END ..........LogRegClassifier__C=0.01;, score=0.703 total time=   2.8s
[CV 3/5] END ..........LogRegClassifier__C=0.01;, score=0.703 total time=   1.6s
[CV 4/5] END ..........LogRegClassifier__C=0.01;, score=0.702 total time=   1.3s
[CV 5/5] END ..........LogRegClassifier__C=0.01;, score=0.709 total time=   1.4s
[CV 1/5] END ...........LogRegClassifier__C=0.1;, score=0.747 total time=   2.4s
[CV 2/5] END ...........LogRegClassifier__C=0.1;, score=0.743 total time=   2.5s
[CV 3/5] END ...........LogRegClassifier__C=0.1;, score=0.743 total time=   2.4s
[CV 4/5] END ...........LogRegClassifier__C=0.1;, score=0.740 total time=   2.6s
[CV 5/5] END ...........LogRegClassifier__C=0.1;, score=0.746 total time=   2.5s
[CV 1/5] END ...........LogRegClassifier__C=1.0;, score=0.759 total time=   3.6s
[CV 2/5] END ...........LogRegClassifier__C=1.0;,

### Αποτελέσματα λογιστικής παλινδρόμησης:
**Best lambda**: 10.000<br>**Best Cross-Val Accuracy**: 0.75<br>**Test Set Accuracy**: 0.74<br>**Test Set F1**: 0.74<br>**Confusion matrix**:

 134    3    0    0    0    0    5    1    0    1    1    0    0    0    1    0    2    1    0    0    1    0    1    2    3    0
   0   93    0    8    0    3    1    2    3    2    0    0    0    0    2    1    1    9   11    0    0    0    0    0    0    0
   0    0  106    0    4    1    9    0    0    0   10    1    1    0    2    0    1    0    0    2    5    0    0    0    0    0
   0   14    0  138    1    0    1    2    0    1    0    0    1    1    2    0    0    4    0    1    0    0    0    1    0    0
   1    4    1    1  111    0    9    0    0    0    0    1    0    0    0    0    2    2    3    3    0    0    0    4    0   10
   0    2    1    3    2  124    1    0    0    0    0    0    0    1    1    4    0    0    0    7    0    0    2    2    3    0
   4    3   24    1    1    1   74    1    0    0    3    2    2    0    1    1   27    6    7    0    0    5    0    1    0    0
   1    1    1    5    0    5    2   72    0    0   10    0    3    8   12    0    1    8 

In [4]:
# classification_report
from sklearn import metrics

print(metrics.classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.85      0.86      0.85       156
           1       0.59      0.68      0.63       136
           2       0.71      0.75      0.73       142
           3       0.73      0.83      0.77       167
           4       0.72      0.73      0.73       152
           5       0.74      0.81      0.78       153
           6       0.57      0.45      0.51       164
           7       0.54      0.48      0.51       151
           8       0.87      0.81      0.84       165
           9       0.81      0.78      0.80       148
          10       0.66      0.65      0.66       146
          11       0.77      0.76      0.76       157
          12       0.85      0.92      0.88       144
          13       0.86      0.80      0.83       166
          14       0.60      0.58      0.59       139
          15       0.88      0.76      0.82       168
          16       0.66      0.70      0.68       168
          17       0.64    

In [5]:
#SVM Γραμμικού Πυρήνα.

from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.svm import SVC

# Training a SVM classifier using SVC class
#svm = SVC(kernel= 'linear', random_state=1, C=1)
#svm.fit(X_train, y_train)
#y_pred = svm.predict(X_test)
# Two steps: a scaler and then a classifier. Declared as tuples
Svm_pipeline=Pipeline([('Scaler',  StandardScaler()),
            ('svc', SVC())])

# defining parameter range
param_grid = {'svc__C': np.logspace(0, 3, 4),
              'svc__kernel': ['linear']}
 
grid = GridSearchCV(Svm_pipeline, param_grid, cv = 5, verbose = 4)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)

display(Markdown(
    "### Αποτελέσματα SVM:\n" +
    "**Best Cross-Val Accuracy**: {:.2f}".format(grid.best_score_) + "<br>" +
    "**Test Set Accuracy**: {:.2f}".format(accuracy_score(y_test, y_pred)) + "<br>" +
    "**Test Set F1**: {:.2f}".format(f1_score(y_test, y_pred,average='micro')) 
))



Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ....svc__C=1.0, svc__kernel=linear;, score=0.835 total time=   4.1s
[CV 2/5] END ....svc__C=1.0, svc__kernel=linear;, score=0.830 total time=   4.0s
[CV 3/5] END ....svc__C=1.0, svc__kernel=linear;, score=0.828 total time=   4.0s
[CV 4/5] END ....svc__C=1.0, svc__kernel=linear;, score=0.820 total time=   3.9s
[CV 5/5] END ....svc__C=1.0, svc__kernel=linear;, score=0.837 total time=   4.1s
[CV 1/5] END ...svc__C=10.0, svc__kernel=linear;, score=0.833 total time=   6.6s
[CV 2/5] END ...svc__C=10.0, svc__kernel=linear;, score=0.825 total time=   6.5s
[CV 3/5] END ...svc__C=10.0, svc__kernel=linear;, score=0.824 total time=   6.5s
[CV 4/5] END ...svc__C=10.0, svc__kernel=linear;, score=0.823 total time=   6.3s
[CV 5/5] END ...svc__C=10.0, svc__kernel=linear;, score=0.839 total time=   6.7s
[CV 1/5] END ..svc__C=100.0, svc__kernel=linear;, score=0.835 total time=  28.6s
[CV 2/5] END ..svc__C=100.0, svc__kernel=linear;,

### Αποτελέσματα SVM:
**Best Cross-Val Accuracy**: 0.83<br>**Test Set Accuracy**: 0.81<br>**Test Set F1**: 0.81

In [6]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87       156
           1       0.67      0.82      0.74       136
           2       0.80      0.82      0.81       142
           3       0.69      0.85      0.76       167
           4       0.76      0.80      0.78       152
           5       0.78      0.90      0.83       153
           6       0.74      0.66      0.70       164
           7       0.62      0.66      0.64       151
           8       0.89      0.84      0.86       165
           9       0.83      0.84      0.84       148
          10       0.71      0.72      0.71       146
          11       0.90      0.84      0.87       157
          12       0.87      0.94      0.90       144
          13       0.94      0.81      0.87       166
          14       0.82      0.68      0.74       139
          15       0.96      0.79      0.87       168
          16       0.75      0.74      0.74       168
          17       0.73    

Μη-γραμμική ταξινόμηση 

In [7]:
#k-Nearest Neighbors

from sklearn.neighbors import KNeighborsClassifier

log_reg_pipeline=Pipeline([('Scaler',  StandardScaler()),
            ('kNNClassifier', KNeighborsClassifier())])

params = { 
           'kNNClassifier__n_neighbors':   range(1,7),
           'kNNClassifier__weights':   ['uniform', 'distance']
         }

grid = GridSearchCV(estimator=log_reg_pipeline, cv = 5, param_grid=params)

grid.fit(X_train, y_train) 
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

display(Markdown(
    "### Αποτελέσματα k-nearest neighbors classification:\n" +
    "**Best k**: {:.3f}".format(grid.best_params_['kNNClassifier__n_neighbors'])  + "<br>" +
    "**Best weighting**: {}".format(grid.best_params_['kNNClassifier__weights'])  + "<br>" +
    "**Best Cross-Val Accuracy**: {:.2f}".format(grid.best_score_) + "<br>" +
    "**Test Set Accuracy**: {:.2f}".format(accuracy_score(y_test, y_test_pred)) + "<br>" +
    "**Test Set F1**: {:.2f}".format(f1_score(y_test, y_test_pred,average='micro')) + "<br>" +
    "**Confusion matrix**:"
))

print("\n".join([' '.join(['{:4}'.format(item) for item in row]) 
      for row in confusion_matrix(y_test,y_test_pred)]))

### Αποτελέσματα k-nearest neighbors classification:
**Best k**: 4.000<br>**Best weighting**: distance<br>**Best Cross-Val Accuracy**: 0.94<br>**Test Set Accuracy**: 0.94<br>**Test Set F1**: 0.94<br>**Confusion matrix**:

 152    0    0    0    3    0    0    0    0    0    0    0    0    0    0    1    0    0    0    0    0    0    0    0    0    0
   0  131    0    0    0    0    0    2    0    0    0    0    0    0    0    0    0    2    1    0    0    0    0    0    0    0
   0    0  136    0    2    0    2    0    0    0    0    0    0    0    2    0    0    0    0    0    0    0    0    0    0    0
   0    2    0  159    0    0    0    4    0    0    0    0    0    0    0    0    0    2    0    0    0    0    0    0    0    0
   0    0    2    0  137    0    5    0    0    0    1    1    0    0    0    0    0    1    0    0    0    0    0    0    0    5
   0    1    0    0    2  142    0    1    0    0    0    0    0    1    0    6    0    0    0    0    0    0    0    0    0    0
   1    1    3    1    1    0  150    1    0    0    0    0    0    0    0    1    3    0    0    1    0    0    0    1    0    0
   0    0    0    7    2    0    1  128    0    0    5    0    1    1    0    0    0    5 

In [8]:
# classification_report
from sklearn import metrics

print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       156
           1       0.89      0.96      0.93       136
           2       0.95      0.96      0.95       142
           3       0.88      0.95      0.92       167
           4       0.90      0.90      0.90       152
           5       0.90      0.93      0.92       153
           6       0.94      0.91      0.93       164
           7       0.83      0.85      0.84       151
           8       0.95      0.97      0.96       165
           9       0.96      0.94      0.95       148
          10       0.89      0.86      0.88       146
          11       0.98      0.97      0.97       157
          12       0.99      0.97      0.98       144
          13       0.97      0.92      0.94       166
          14       0.94      0.94      0.94       139
          15       0.95      0.90      0.92       168
          16       0.94      0.93      0.94       168
          17       0.91    

In [12]:
#RBF SVM

# Two steps: a scaler and then a classifier. Declared as tuples
Svm_pipeline=Pipeline([('Scaler',  StandardScaler()),
            ('svc', SVC())])

# defining parameter range
param_grid = {'svc__C': np.logspace(0, 3, 4),
              'svc__gamma': np.logspace(-1,1,6),
              'svc__kernel': ['rbf']}
 
grid = GridSearchCV(Svm_pipeline, param_grid, cv = 5, verbose = 4)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)

display(Markdown(
    "### Αποτελέσματα SVM:\n" +
    "**Best Cross-Val Accuracy**: {:.2f}".format(grid.best_score_) + "<br>" +
    "**Test Set Accuracy**: {:.2f}".format(accuracy_score(y_test, y_pred)) + "<br>" +
    "**Test Set F1**: {:.2f}".format(f1_score(y_test, y_pred,average='micro')) 
))











Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END svc__C=1.0, svc__gamma=0.1, svc__kernel=rbf;, score=0.942 total time=   6.7s
[CV 2/5] END svc__C=1.0, svc__gamma=0.1, svc__kernel=rbf;, score=0.935 total time=   6.3s
[CV 3/5] END svc__C=1.0, svc__gamma=0.1, svc__kernel=rbf;, score=0.933 total time=   6.5s
[CV 4/5] END svc__C=1.0, svc__gamma=0.1, svc__kernel=rbf;, score=0.924 total time=   6.3s
[CV 5/5] END svc__C=1.0, svc__gamma=0.1, svc__kernel=rbf;, score=0.939 total time=   6.3s
[CV 1/5] END svc__C=1.0, svc__gamma=0.251188643150958, svc__kernel=rbf;, score=0.958 total time=   9.3s
[CV 2/5] END svc__C=1.0, svc__gamma=0.251188643150958, svc__kernel=rbf;, score=0.953 total time=   9.3s
[CV 3/5] END svc__C=1.0, svc__gamma=0.251188643150958, svc__kernel=rbf;, score=0.956 total time=   9.5s
[CV 4/5] END svc__C=1.0, svc__gamma=0.251188643150958, svc__kernel=rbf;, score=0.947 total time=   9.2s
[CV 5/5] END svc__C=1.0, svc__gamma=0.251188643150958, svc__kernel=rbf;,

### Αποτελέσματα SVM:
**Best Cross-Val Accuracy**: 0.96<br>**Test Set Accuracy**: 0.97<br>**Test Set F1**: 0.97

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       156
           1       0.92      0.97      0.95       136
           2       0.97      0.99      0.98       142
           3       0.93      0.98      0.95       167
           4       0.96      0.92      0.94       152
           5       0.94      0.93      0.94       153
           6       0.97      0.98      0.97       164
           7       0.91      0.90      0.91       151
           8       0.98      0.96      0.97       165
           9       0.95      0.97      0.96       148
          10       0.94      0.94      0.94       146
          11       0.99      0.96      0.97       157
          12       0.97      0.99      0.98       144
          13       0.98      0.95      0.97       166
          14       0.96      0.99      0.98       139
          15       0.96      0.96      0.96       168
          16       0.99      0.97      0.98       168
          17       0.96    